In [1]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, RegressorMixin
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 
import itertools
from datetime import datetime

In [23]:
dataset=pd.read_csv("iris-virginica.csv")
df=dataset[100:]
df=df.append(dataset[0:100])

In [ ]:
X = df.columns[1:]
X = df[X]
X = np.array(X)
X=X.astype(float)

y = df.columns[0]
y = df[y]
y = np.array(y)
y=y.astype(float)
y=np.where(y==0,-1,y)

In [24]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma):
    # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

In [25]:
from cvxopt import matrix
class HYP_SVM(BaseEstimator, RegressorMixin):
    
    def __init__(self, kernel=gaussian_kernel, sigma=None, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
        self.sigma = sigma
        
    #geometric mean
    def gm(self, y_predict, y_test):
        print(y_predict)
        test_min=0
        test_max=0
        pred_min=0
        pred_max=0
        y_test=np.asarray(y_test)
        for i in range(0,len(y_test)):
            if(y_test[i]==1):
                test_min=test_min+1
            else:
                test_max=test_max+1
        #print("y_test min",test_min)       
        #print("y_test max",test_max)
        for i in range(0,len(y_predict)):
            if(y_predict[i]==1 and y_predict[i]==y_test[i]):
                pred_min=pred_min+1
            elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
                pred_max=pred_max+1
        #print("y_pred min",pred_min)       
        #print("y_pred max",pred_max)
        se=pred_min/test_min
        sp=pred_max/test_max
        #print(se,sp)
        gm=math.sqrt(se*sp)
        print("GM",gm) 
        return gm
        
    def score(self, X, y):
        y_predict=self.predict(X)
        #gm=self.gm(y_predict, y)
        correct = np.sum(y_predict == y)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        #mse=mean_squared_error(y, y_predict, squared=False)
        acc=correct/len(y_predict)
        print("Accuracy",correct/len(y_predict))
        #print("Errore quadratico medio: ", mse)
        return acc
    
    def accuracy(self, X, y):
        y_predict=self.predict(X)
        correct = np.sum(y_predict == y)
        acc=correct/len(y_predict)
        return acc
          
    def get_params(self, deep=True):
        return {"C": self.C, "sigma": self.sigma}        
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def m_func(self, X_train, y, decaying_func="linear"):
        n_samples, n_features = X_train.shape
        pos = len(y[y==1])
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        X_train=np.asarray(X_train)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        #print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)

        if(decaying_func=="linear"):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(decaying_func=="exponential"):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=pos/n_samples
        r_min=1
        self.m=func[0:pos]*r_min
        #print(self.m.shape)
        self.m=np.append(self.m,func[pos:n_samples]*r_max)
        #print(self.m.shape)
        
 ##############################################################################

    #prendeva come argomento anche x_test, l'ho tolto, ho aggiunto K
    def fit(self, X_train, y, decaying_func="linear"):
        
        self.m_func(X_train, y, decaying_func)
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 

        # Gram matrix

        #print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        #print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None
            
        return self    
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv, self.sigma)
                y_predict[i] = s
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))
    

In [26]:
C_vals = [1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4]
sigma = [9e-2, 9e-1, 9, 9e+1, 9e+2, 9e+3, 9e+4]
parameters = {'C': C_vals, 'sigma': sigma}
#err = []
acc = []
#metto shuffle a False, è richiesto nella classe di indicare la suddivisione nel training set di dati positivi e negativi
skf = StratifiedKFold(n_splits=5, shuffle=False)
print(datetime.now())
for train_index, validation_index in skf.split(X, y):
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y[train_index], y[validation_index]
    
    model = HYP_SVM(C=100, kernel=gaussian_kernel, sigma=0.9)
    clf = GridSearchCV(model, parameters, cv=5)
    grid_result = clf.fit(X=X_train, y=y_train)
    best_params = clf.best_params_
    best_C = best_params['C']
    best_sigma = best_params['sigma']
    print("\nBest model", best_C, best_sigma)
    best_model = HYP_SVM(C=best_C, kernel=gaussian_kernel, sigma=best_sigma)
    best_model.fit(X_train, y_train)
    #err.append(best_model.score(X_validation, y_validation))
    acc.append(best_model.score(X_validation, y_validation))
print(datetime.now())    

2020-10-03 17:25:27.581646
     pcost       dcost       gap    pres   dres
 0: -1.4706e+01 -4.7135e+00  5e+02  2e+01  3e-16
 1: -2.1560e+00 -1.8077e+00  2e+01  1e+00  4e-16
 2: -3.8744e-01 -1.5073e+00  2e+00  3e-02  1e-15
 3: -3.1799e-01 -4.3783e-01  1e-01  7e-17  5e-15
 4: -3.1895e-01 -3.2047e-01  2e-03  4e-17  3e-16
 5: -3.1900e-01 -3.1915e-01  1e-04  4e-17  2e-16
 6: -3.1901e-01 -3.1902e-01  5e-06  5e-17  2e-16
 7: -3.1901e-01 -3.1901e-01  1e-07  2e-17  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.4706e+01 -4.7135e+00  5e+02  2e+01  3e-16
 1: -2.1560e+00 -1.8077e+00  2e+01  1e+00  4e-16
 2: -3.8744e-01 -1.5073e+00  2e+00  3e-02  1e-15
 3: -3.1799e-01 -4.3783e-01  1e-01  7e-17  5e-15
 4: -3.1895e-01 -3.2047e-01  2e-03  4e-17  3e-16
 5: -3.1900e-01 -3.1915e-01  1e-04  4e-17  2e-16
 6: -3.1901e-01 -3.1902e-01  5e-06  5e-17  2e-16
 7: -3.1901e-01 -3.1901e-01  1e-07  2e-17  2e-16
Optimal solution found.
0 out of 24 predictions correct
Accuracy 0.0


     pcost       dcost       gap    pres   dres
 0: -8.3573e+00 -4.3349e+00  6e+02  2e+01  7e-16
 1: -3.1879e+00 -3.1020e+00  3e+01  1e+00  6e-16
 2: -9.2156e-01 -2.1118e+00  3e+00  6e-02  2e-15
 3: -7.3417e-01 -1.0230e+00  3e-01  9e-17  1e-15
 4: -7.4930e-01 -7.9523e-01  5e-02  2e-16  6e-16
 5: -7.5634e-01 -7.6327e-01  7e-03  2e-17  5e-16
 6: -7.5803e-01 -7.5938e-01  1e-03  4e-17  5e-16
 7: -7.5839e-01 -7.5877e-01  4e-04  1e-16  4e-16
 8: -7.5851e-01 -7.5863e-01  1e-04  2e-16  4e-16
 9: -7.5856e-01 -7.5857e-01  1e-05  2e-16  5e-16
10: -7.5856e-01 -7.5856e-01  2e-07  1e-16  5e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -8.3573e+00 -4.3349e+00  6e+02  2e+01  7e-16
 1: -3.1879e+00 -3.1020e+00  3e+01  1e+00  6e-16
 2: -9.2156e-01 -2.1118e+00  3e+00  6e-02  2e-15
 3: -7.3417e-01 -1.0230e+00  3e-01  9e-17  1e-15
 4: -7.4930e-01 -7.9523e-01  5e-02  2e-16  6e-16
 5: -7.5634e-01 -7.6327e-01  7e-03  2e-17  5e-16
 6: -7.5803e-01 -7.5938e-01  1e-03  4e-17  5e-1

 6: -7.9396e-01 -7.9441e-01  5e-04  1e-07  6e-16
 7: -7.9413e-01 -7.9420e-01  7e-05  1e-08  5e-16
 8: -7.9416e-01 -7.9418e-01  1e-05  2e-09  5e-16
 9: -7.9417e-01 -7.9417e-01  1e-06  1e-11  4e-16
10: -7.9417e-01 -7.9417e-01  1e-08  1e-13  5e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.4804e+01 -4.8307e+00  5e+02  2e+01  2e-15
 1: -5.0649e+00 -2.6943e+00  3e+01  1e+00  2e-15
 2: -7.5119e-01 -1.9432e+00  1e+00  1e-03  2e-15
 3: -7.8762e-01 -8.4290e-01  6e-02  5e-05  7e-16
 4: -7.8992e-01 -8.0076e-01  1e-02  9e-06  5e-16
 5: -7.9281e-01 -7.9813e-01  5e-03  1e-06  5e-16
 6: -7.9396e-01 -7.9441e-01  5e-04  1e-07  6e-16
 7: -7.9413e-01 -7.9420e-01  7e-05  1e-08  5e-16
 8: -7.9416e-01 -7.9418e-01  1e-05  2e-09  5e-16
 9: -7.9417e-01 -7.9417e-01  1e-06  1e-11  4e-16
10: -7.9417e-01 -7.9417e-01  1e-08  1e-13  5e-16
Optimal solution found.
24 out of 24 predictions correct
Accuracy 1.0
     pcost       dcost       gap    pres   dres
 0: -2.6708e+01 -2.0982e+0

 3: -7.9952e-01 -8.5621e-01  6e-02  1e-16  6e-16
 4: -7.9999e-01 -8.0056e-01  6e-04  1e-16  6e-16
 5: -8.0000e-01 -8.0001e-01  7e-06  5e-17  1e-15
 6: -8.0000e-01 -8.0000e-01  1e-06  2e-16  5e-16
 7: -8.0000e-01 -8.0000e-01  5e-07  7e-17  9e-16
Optimal solution found.
24 out of 24 predictions correct
Accuracy 1.0
     pcost       dcost       gap    pres   dres
 0: -4.7123e+01 -1.9908e+00  4e+02  2e+01  4e-15
 1: -1.1494e+00 -1.9791e+00  5e+00  2e-01  3e-15
 2: -7.5209e-01 -1.4188e+00  7e-01  3e-17  7e-16
 3: -7.9952e-01 -8.5621e-01  6e-02  6e-17  5e-16
 4: -7.9999e-01 -8.0056e-01  6e-04  6e-17  5e-16
 5: -8.0000e-01 -8.0001e-01  7e-06  6e-17  9e-16
 6: -8.0000e-01 -8.0000e-01  1e-06  6e-17  4e-16
 7: -8.0000e-01 -8.0000e-01  2e-07  1e-16  9e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.7123e+01 -1.9908e+00  4e+02  2e+01  4e-15
 1: -1.1494e+00 -1.9791e+00  5e+00  2e-01  3e-15
 2: -7.5209e-01 -1.4188e+00  7e-01  3e-17  7e-16
 3: -7.9952e-01 -8.5621e-0

     pcost       dcost       gap    pres   dres
 0: -4.7133e+01 -1.9873e+00  4e+02  2e+01  3e-15
 1: -1.1495e+00 -1.9791e+00  5e+00  2e-01  3e-15
 2: -7.5209e-01 -1.4188e+00  7e-01  2e-16  6e-16
 3: -7.9952e-01 -8.5621e-01  6e-02  3e-17  5e-16
 4: -7.9999e-01 -8.0056e-01  6e-04  3e-17  5e-16
 5: -8.0000e-01 -8.0001e-01  6e-06  3e-16  1e-15
 6: -8.0000e-01 -8.0000e-01  6e-08  6e-16  1e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.7133e+01 -1.9873e+00  4e+02  2e+01  3e-15
 1: -1.1495e+00 -1.9791e+00  5e+00  2e-01  3e-15
 2: -7.5209e-01 -1.4188e+00  7e-01  2e-16  6e-16
 3: -7.9952e-01 -8.5621e-01  6e-02  3e-17  5e-16
 4: -7.9999e-01 -8.0056e-01  6e-04  3e-17  5e-16
 5: -8.0000e-01 -8.0001e-01  6e-06  3e-16  1e-15
 6: -8.0000e-01 -8.0000e-01  6e-08  6e-16  1e-15
Optimal solution found.
24 out of 24 predictions correct
Accuracy 1.0
     pcost       dcost       gap    pres   dres
 0: -4.7133e+01 -1.9873e+00  4e+02  2e+01  3e-15
 1: -1.1495e+00 -1.9791e+00

...

     pcost       dcost       gap    pres   dres
 0:  2.5409e+08 -1.8211e+09  3e+09  6e-02  6e-12
 1:  9.1431e+07 -1.8993e+08  3e+08  5e-03  2e-10
 2:  2.2114e+07 -3.6158e+07  6e+07  7e-04  3e-11
 3:  3.5020e+06 -5.0268e+06  9e+06  9e-12  2e-12
 4:  5.0751e+05 -5.9329e+05  1e+06  2e-12  7e-13
 5:  7.2221e+04 -8.3689e+04  2e+05  8e-13  2e-13
 6:  1.0023e+04 -1.2185e+04  2e+04  2e-13  1e-13
 7:  1.2744e+03 -1.8827e+03  3e+03  2e-13  5e-14
 8:  1.2583e+02 -4.3981e+02  6e+02  1e-13  2e-14
 9: -9.3907e+01 -2.2644e+02  1e+02  2e-14  1e-14
10: -1.4431e+02 -1.6417e+02  2e+01  5e-14  1e-14
11: -1.5254e+02 -1.5519e+02  3e+00  4e-14  2e-14
12: -1.5378e+02 -1.5394e+02  2e-01  7e-14  1e-14
13: -1.5388e+02 -1.5389e+02  5e-03  5e-14  1e-14
14: -1.5388e+02 -1.5388e+02  1e-04  7e-14  1e-14
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.5409e+08 -1.8211e+09  3e+09  6e-02  6e-12
 1:  9.1431e+07 -1.8993e+08  3e+08  5e-03  2e-10
 2:  2.2114e+07 -3.6158e+07  6e+07  7e-04  3e-1

     pcost       dcost       gap    pres   dres
 0:  3.0357e+08 -2.9242e+09  3e+09  2e-03  2e-11
 1:  9.3408e+07 -2.4864e+08  3e+08  1e-04  2e-11
 2:  1.9425e+07 -3.6359e+07  6e+07  1e-05  1e-11
 3:  3.7653e+06 -8.8240e+06  1e+07  6e-07  8e-12
 4:  9.6427e+05 -2.1892e+06  3e+06  7e-08  6e-12
 5:  1.4732e+05 -3.0570e+05  5e+05  1e-12  4e-12
 6:  3.3703e+04 -1.2780e+05  2e+05  6e-12  2e-12
 7:  6.9478e+03 -9.3052e+04  1e+05  8e-12  3e-12
 8: -1.2663e+04 -7.2484e+04  6e+04  1e-11  3e-12
 9: -2.0074e+04 -6.6711e+04  5e+04  4e-12  4e-12
10: -2.8634e+04 -4.9102e+04  2e+04  6e-12  5e-12
11: -3.5549e+04 -3.8561e+04  3e+03  1e-12  5e-12
12: -3.6872e+04 -3.6931e+04  6e+01  2e-13  6e-12
13: -3.6908e+04 -3.6909e+04  6e-01  1e-11  5e-12
14: -3.6909e+04 -3.6909e+04  6e-03  1e-11  5e-12
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  3.0357e+08 -2.9242e+09  3e+09  2e-03  2e-11
 1:  9.3408e+07 -2.4864e+08  3e+08  1e-04  2e-11
 2:  1.9425e+07 -3.6359e+07  6e+07  1e-05  1e-1

     pcost       dcost       gap    pres   dres
 0:  3.6471e+07 -7.8426e+08  8e+08  7e-12  3e-11
 1:  2.5087e+07 -7.7052e+07  1e+08  1e-10  3e-11
 2:  6.0146e+06 -2.1589e+07  3e+07  1e-10  3e-11
 3:  1.3102e+06 -3.2262e+06  5e+06  9e-12  2e-11
 4:  2.1809e+05 -7.2640e+05  9e+05  3e-11  1e-11
 5:  6.2517e+04 -4.7535e+05  5e+05  1e-11  1e-11
 6: -4.3619e+04 -2.1283e+05  2e+05  4e-12  8e-12
 7: -7.6325e+04 -1.9167e+05  1e+05  3e-12  1e-11
 8: -9.9203e+04 -1.6359e+05  6e+04  8e-12  2e-11
 9: -1.1375e+05 -1.4440e+05  3e+04  3e-11  1e-11
10: -1.2148e+05 -1.3247e+05  1e+04  7e-11  2e-11
11: -1.2437e+05 -1.2945e+05  5e+03  2e-11  2e-11
12: -1.2644e+05 -1.2685e+05  4e+02  6e-11  2e-11
13: -1.2663e+05 -1.2663e+05  7e+00  4e-11  2e-11
14: -1.2663e+05 -1.2663e+05  7e-02  3e-11  2e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  3.6471e+07 -7.8426e+08  8e+08  7e-12  3e-11
 1:  2.5087e+07 -7.7052e+07  1e+08  1e-10  3e-11
 2:  6.0146e+06 -2.1589e+07  3e+07  1e-10  3e-1

19 out of 24 predictions correct
Accuracy 0.7916666666666666
     pcost       dcost       gap    pres   dres
 0:  1.8887e+05 -6.6874e+08  7e+08  1e-11  3e-11
 1:  1.8663e+05 -8.1476e+06  8e+06  7e-12  3e-11
 2:  6.5350e+04 -1.2746e+06  1e+06  6e-11  3e-11
 3:  2.4678e+04 -1.2466e+06  1e+06  8e-11  3e-11
 4: -4.1809e+04 -1.1333e+06  1e+06  2e-12  3e-11
 5: -9.7129e+04 -8.9426e+05  8e+05  5e-12  3e-11
 6: -1.5554e+05 -5.4618e+05  4e+05  2e-11  3e-11
 7: -1.9177e+05 -2.5751e+05  7e+04  3e-11  2e-11
 8: -1.9968e+05 -2.2850e+05  3e+04  3e-11  2e-11
 9: -2.0102e+05 -2.2652e+05  3e+04  3e-11  2e-11
10: -2.0411e+05 -2.2563e+05  2e+04  5e-11  2e-11
11: -2.0865e+05 -2.1800e+05  9e+03  8e-11  2e-11
12: -2.1056e+05 -2.1463e+05  4e+03  7e-11  2e-11
13: -2.1207e+05 -2.1269e+05  6e+02  5e-12  3e-11
14: -2.1230e+05 -2.1241e+05  1e+02  5e-12  3e-11
15: -2.1234e+05 -2.1235e+05  8e+00  7e-12  3e-11
16: -2.1235e+05 -2.1235e+05  2e-01  5e-11  2e-11
17: -2.1235e+05 -2.1235e+05  3e-03  1e-10  3e-11
Optimal s

     pcost       dcost       gap    pres   dres
 0: -2.6668e+05 -1.0687e+09  1e+09  4e-11  2e-11
 1: -2.6671e+05 -1.1004e+07  1e+07  8e-11  2e-11
 2: -2.6924e+05 -4.2692e+05  2e+05  7e-11  2e-11
 3: -3.1567e+05 -3.2122e+05  6e+03  1e-10  3e-11
 4: -3.1982e+05 -3.2003e+05  2e+02  1e-10  3e-11
 5: -3.1997e+05 -3.2004e+05  6e+01  4e-11  3e-11
 6: -3.1999e+05 -3.2000e+05  9e+00  4e-11  3e-11
 7: -3.2000e+05 -3.2000e+05  9e-01  3e-11  4e-11
 8: -3.2000e+05 -3.2000e+05  9e-02  2e-10  4e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.6668e+05 -1.0687e+09  1e+09  4e-11  2e-11
 1: -2.6671e+05 -1.1004e+07  1e+07  8e-11  2e-11
 2: -2.6924e+05 -4.2692e+05  2e+05  7e-11  2e-11
 3: -3.1567e+05 -3.2122e+05  6e+03  1e-10  3e-11
 4: -3.1982e+05 -3.2003e+05  2e+02  1e-10  3e-11
 5: -3.1997e+05 -3.2004e+05  6e+01  4e-11  3e-11
 6: -3.1999e+05 -3.2000e+05  9e+00  4e-11  3e-11
 7: -3.2000e+05 -3.2000e+05  9e-01  3e-11  4e-11
 8: -3.2000e+05 -3.2000e+05  9e-02  2e-10  4e-1

In [27]:
print(np.mean(acc))

0.9666666666666666


In [28]:
print(acc)

[0.9666666666666667, 1.0, 1.0, 0.9, 0.9666666666666667]
